# Curated DLT Pipeline

Coaches Dlt Pipeline

In [ ]:
import dlt
from pyspark.sql.functions import *

#### Coaches Expectations

In [ ]:
#Expectations

coach_expect={
    "no_missing_records":"code is not null",
    "current_status":"current is true"
}

In [ ]:
#Streaming_table

@dlt.table
@dlt.expect_all(coach_expect)
def source_coaches():
  df=spark.readStream.table("olympiccatalog.silver_schema.coaches")
  return df

In [ ]:
#View

@dlt.view
def coaches_view():
    df=spark.read.table("LIVE.source_coaches")
    return df

In [ ]:
#Materialized_view

@dlt.table
@dlt.expect_all(coach_expect)
def couches_table():
    df = spark.read.table("LIVE.coaches_view")
    return df

## nocs table

In [ ]:
#Expectations

nocs_expect={
    "nocs_no_missing_records":"code is not null"
      }

In [ ]:
#View

@dlt.view
def nocs_view():
  df=spark.readStream.table("olympiccatalog.silver_schema.nocs")
  return df

In [ ]:
#Streaming table

@dlt.table
@dlt.expect_all_or_drop(nocs_expect)
def nocs_table():
    df=spark.readStream.table("LIVE.nocs_view")
    return df

## Events Table

In [ ]:
#Expectations

events_expect={
    "nocs_no_missing_records":"event is not null"
      }

In [ ]:
#View

@dlt.view
def events_view():
  df=spark.readStream.table("olympiccatalog.silver_schema.events")
  return df

In [ ]:
#Streaming table

@dlt.table
@dlt.expect_all(events_expect)
def event_table():
    df=spark.readStream.table("LIVE.events_view")
    return df

## Athletes CDC

In [ ]:
#View

@dlt.view
def athletes_view():
  df=spark.readStream.table("olympiccatalog.silver_schema.athletes")
  return df

In [ ]:
#Empty streaming table

dlt.create_streaming_table("athletes_target")

#### change data feed in dlt using apply changes api

In [ ]:
#
dlt.apply_changes(
  target = "athletes_target",
  source = "athletes_view",
  keys = ["athlete_id"],
  sequence_by = col("height"),
  stored_as_scd_type = 1
)